## List of All dataset

In [6]:
from datasets import load_dataset
import os
#os.environ["TOKENIZERS_PARALLELISM"] = "false" ##I will find a way to fix this later :(
#os.environ["TOKENIZERS_PARALLELISM"] = "true" ##I will find a way to fix this later :(


import torch
from torch.utils.data import TensorDataset
from torch.utils.data import random_split
from torch.utils.data import RandomSampler,SequentialSampler
from torch.utils.data import DataLoader

from transformers import AutoTokenizer
from transformers import AutoModelForMaskedLM
from transformers import AutoConfig
from transformers import DataCollatorForLanguageModeling

In [7]:
def getWiki(DIR):
    
    cache_dir=DIR
    
    dataset=load_dataset('wikipedia', '20200501.en', beam_runner='DirectRunner', cache_dir=cache_dir)
    
    return dataset

In [8]:
def test():
    
    if os.uname()[1].find('gilbreth')==0: ##if not darwin(mac/locallaptop)
        DIR='/scratch/gilbreth/das90/Dataset'
    elif os.uname()[1].find('unimodular')==0:
        DIR='/scratch2/das90/Dataset'
    elif os.uname()[1].find('Siddharthas')==0:
        DIR='/Users/siddharthashankardas/Purdue/Dataset'  
    else:
        DIR='./Results'
        
    cache_dir=DIR
    
    dataset=getWiki(cache_dir)
    
    return dataset

In [9]:
dataset=test()

Downloading:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.87k [00:00<?, ?B/s]

Old caching folder /scratch/gilbreth/das90/Dataset/wikipedia/20200501.en/1.0.0/2fe8db1405aef67dff9fcc51e133e1f9c5b0106f9d9e9638188176d278fd5ff1 for dataset wikipedia exists but not data were found. Removing it. 


Downloading:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.3G [00:00<?, ?B/s]

Dataset wikipedia downloaded and prepared to /scratch/gilbreth/das90/Dataset/wikipedia/20200501.en/1.0.0/2fe8db1405aef67dff9fcc51e133e1f9c5b0106f9d9e9638188176d278fd5ff1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
dataset['train'].shape

(6078422, 2)

In [11]:
dataset['train']

Dataset({
    features: ['title', 'text'],
    num_rows: 6078422
})

In [12]:
def encode(examples):
    return berttokenizer(examples['text'], truncation=True)

class WikiDataset(torch.utils.data.Dataset):
    def __init__(self, berttokenizer, collator, dataset):
        self.tokenizer = berttokenizer
        self.dataset = dataset
        self.collator = collator

    def __getitem__(self, idx):
        item = encode(dataset['train'][idx])        
        item = self.collator([item])
        item = {key: val[0] for key, val in item.items()}
    
        return item

    def __len__(self):
        return dataset.num_rows
        
A=AutoTokenizer
berttokenizer=A.from_pretrained('bert-base-uncased')
datacollator=DataCollatorForLanguageModeling(tokenizer=berttokenizer,mlm_probability=0.15, mlm=True)

wikidataset = WikiDataset(berttokenizer,datacollator, dataset)

wikidataset[4]

{'input_ids': tensor([  101,  2073,  2003,   103,  1029,  1006,  1007,  2003,  1037,  4051,
          3059,  4038,  1011,  3689, 28026,  2856,  2011, 10704,  5811, 13348,
          3490,  1012,  1996,  2143,  2018,  1037, 11587,  2537,  2138,  1010,
          2044,  5008,  2070,  5019,  1010,  5811, 13348,  3490,  2439,  3037,
          1999,  1996,  2143,  1998,  4704,  1996,  2275,  1012,  1996,  2147,
          2001,  2949,  2044,   103,  1037,  2095,  1010,  3701,   103,  7986,
         12256,  6610,   103,   103,  2007,  2070,  5019,  2036,  2915,  2011,
         12776,  3695, 11865, 15472,  2072,  1998, 20493,   103,  5498,   103,
          2750,  2008,  1010,  5811, 27843,  3490,  2003,  1996,  7082,  5827,
          2472,  1997,  1996,  2143,  1012,  5436,  8190,  1998, 13460,  1997,
          2019,  4654,  1011, 20462,  1012,  7861, 16313, 14050,  1998,  4487,
         27572, 24117,  2098,  2011,  2166,   103,  2002,  2097,  2574,  2933,
          2010,   103,  2000,  3827,  1

In [13]:
dataset.num_rows['train']

6078422